In [1]:
from typing import List, Dict, Set, Any, Optional, Tuple, Literal, Callable
import time

import numpy as np
import jax
import jax.numpy as jnp
import jax.lax as lax
from jaxtyping import Array, Float, Int, PRNGKeyArray
import aeon
import pandas as pd

from features.sig_trp import SigVanillaTensorizedRandProj, SigRBFTensorizedRandProj
from features.sig import SigTransform, LogSigTransform
from features.base import TimeseriesFeatureTransformer, TabularTimeseriesFeatures, RandomGuesser
from features.sig_neural import RandomizedSignature
from utils import print_name, print_shape

from preprocessing.timeseries_augmentation import normalize_mean_std_traindata, normalize_streams, augment_time, add_basepoint_zero
from aeon.classification.sklearn import RotationForestClassifier
from sklearn.metrics import root_mean_squared_error

jax.config.update('jax_platform_name', 'cpu') # Used to set the platform (cpu, gpu, etc.)
np.set_printoptions(precision=3, threshold=5) # Print options

from aeon.datasets.tser_datasets import tser_soton
from aeon.datasets import load_regression

from sklearn.linear_model import RidgeCV
from aeon.transformations.collection.convolution_based import Rocket, MultiRocketMultivariate, MiniRocketMultivariate

from run_tser import do_experiments, get_aeon_dataset

/home/nikita/Code/zephyrox/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# print(len(tser_soton))

# for dataset_name in list(tser_soton)[10:]:
#     print(dataset_name)
#     X_train, y_train, X_test, y_test = get_aeon_dataset(dataset_name)
#     print_shape(X_train)
#     print_name(y_train)
#     print("\n")

In [3]:
#run experiments
d_res, d_meta, d_failed = do_experiments(["BitcoinSentiment", "AluminiumConcentration"])

# make dict of results
model_names = d_res[list(d_res.keys())[0]][0][0]
alpha_names = ["alpha_" + model_name for model_name in model_names]

df_RMSEs_test = pd.DataFrame({dataset : RMSEs_test for dataset, ((model_names, RMSEs_test, RMSEs_train, times_trans, times_fit, alphas),) in d_res.items()}).transpose()
df_RMSEs_test.columns = model_names
df_RMSEs_train = pd.DataFrame({dataset : RMSEs_train for dataset, ((model_names, RMSEs_test, RMSEs_train, times_trans, times_fit, alphas),) in d_res.items()}).transpose()
df_RMSEs_train.columns = model_names
df_trans = pd.DataFrame({dataset : times_trans for dataset, ((model_names, RMSEs_test, RMSEs_train, times_trans, times_fit, alphas),) in d_res.items()}).transpose()
df_trans.columns = model_names
df_fit = pd.DataFrame({dataset : times_fit for dataset, ((model_names, RMSEs_test, RMSEs_train, times_trans, times_fit, alphas),) in d_res.items()}).transpose()
df_fit.columns = model_names
df_alphas = pd.DataFrame({dataset : alphas for dataset, ((model_names, RMSEs_test, RMSEs_train, times_trans, times_fit, alphas),) in d_res.items()}).transpose()
df_alphas.columns = alpha_names

meta = pd.DataFrame(d_meta).transpose()

df_RMSEs_test = pd.concat([meta, df_RMSEs_test], axis=1)
df_RMSEs_train = pd.concat([meta, df_RMSEs_train], axis=1)
df_trans = pd.concat([meta, df_trans], axis=1)
df_fit = pd.concat([meta, df_fit], axis=1)
df_alphas = pd.concat([meta, df_alphas], axis=1)

# save
df_RMSEs_test.to_pickle("df_RMSEs_test_TSER.pkl")
df_RMSEs_train.to_pickle("df_RMSEs_train_TSER.pkl")
df_trans.to_pickle("df_trans_TSER.pkl")
df_fit.to_pickle("df_fit_TSER.pkl")
df_alphas.to_pickle("df_alphas_TSER.pkl")
print(d_failed)

BitcoinSentiment
Elapsed time 11.064262390136719
AluminiumConcentration


ValueError: n_timepoints must be >= 9, but found 3; zero pad shorter series so that n_timepoints == 9

In [ ]:
df_RMSEs_test = pd.read_pickle("df_RMSEs_test_TSER.pkl")
df_RMSEs_test

In [ ]:
df_RMSEs_test.mean()

In [ ]:
df_RMSEs_train = pd.read_pickle("df_RMSEs_train_TSER.pkl")
df_RMSEs_train

In [ ]:
df_RMSEs_train.mean()

In [ ]:
df_trans = pd.read_pickle("df_trans_TSER.pkl")
df_trans

In [ ]:
df_trans.mean()

In [ ]:
df_fit = pd.read_pickle("df_fit_TSER.pkl")
df_fit

In [ ]:
df_fit.mean()

In [ ]:
df_alphas = pd.read_pickle("df_alphas_TSER.pkl")
df_alphas

In [ ]:
df_alphas.mean()